# 多线程获取股票数据
# 接口很容易被封

In [ ]:
import akshare as ak
import pandas as pd
import threading


# 获取股票代码
# 获取股票代码
df_sz = ak.stock_info_sz_name_code()
df_sh = ak.stock_info_sh_name_code()

In [ ]:
df_sz['A股代码'] = 'sz' + df_sz['A股代码']
df_sh['公司代码'] = 'sh' + df_sh['公司代码']
print(df_sz)
print(df_sh)

In [ ]:
# df_sz[df_sz['板块']=='主板']
# df_sz['板块'].unique()
# df_zb = pd.DataFrame(columns='')
sz_colume = ['板块', '代码', '简称', '上市日期', '总股本', '流通股本', '所属行业']
sh_colume = ['代码', '简称', '公司代码', '公司简称', '上市日期']
df_sz.columns = sz_colume
df_sh.columns = sh_colume
print(df_sz.columns)
print(df_sh.columns)

In [ ]:
df_sh['板块'] = '主板'

In [ ]:
print(df_sh.columns)

In [ ]:
sock_all = pd.DataFrame()
sock_all = sock_all.append(df_sz)
sock_all = sock_all.append(df_sh[['板块', '代码', '简称', '上市日期']])
print(sock_all.shape)
print(sock_all.tail())

In [ ]:
sock_all = sock_all.reset_index(drop=True)
print(sock_all.tail())
sock_all.to_csv('/Users/jinkun.tian/gitee/financial_lab/AKshareTest/source/sock_all.csv', index=False)

In [ ]:
# 多线程下载，每个线程下载500条记录。

# 启动7个线程，下载
# 将7个df合并

In [ ]:
error_scok_code_list = []
def get_sock_df(sc_df, start_index, count, start_time, end_time):
    out_df = pd.DataFrame()
    for i in range(count):
        try:
            code = sc_df.loc[start_index + i][['代码','简称']]
        except:
            print("代码获取完毕:", start_index + i)
        try:
            df = ak.stock_zh_a_daily(code['代码'], start_time, end_time, adjust="qfq")
        except:
            print('error')
            print("获取失败",code)
            error_scok_code_list.append(code)
            continue
        df['code'] = code['代码']
        df['name'] = code['简称']
        out_df = out_df.append(df)
    return out_df

In [ ]:
class myThread(threading.Thread):
    def __init__(self, threadID, sc_df, start_index, count, start_time, end_time):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.sc_df = sc_df
        self.start_index = start_index
        self.start_time = start_time
        self.end_time = end_time
        self.count = count
        self.out_df = None

    def run(self):
        print("开启线程：" + str(self.threadID))
        self.out_df = get_sock_df(self.sc_df, self.start_index, self.count, self.start_time, self.end_time)
        print("退出线程：" + str(self.threadID))

In [ ]:
start_time = '20180601'
end_time = '20180801'
count = 500
thread_list = list()
zb_sock_all = sock_all[sock_all['板块']=='主板']
zb_sock_all = zb_sock_all.reset_index(drop=True)
for i in range(7):
    start = 500*i
    thread = myThread(i,zb_sock_all,start,count,start_time,end_time)
    thread_list.append(thread)
    thread.start()

for i in thread_list:
    thread.join()

In [ ]:
for thread in thread_list:
    print(thread.threadID,thread.start_index,thread.count,thread.out_df.shape)
